# Table of Contents
#### 01. Importing
#### 02. Wrangling
#### 03. Data Quality and Consistency Checks
#### 04. Merging Data
#### 05. Export Data

# 01. 
Importing

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Path
path =r'/Users/heathervisentin/Desktop/Data Analysis/Achievement 4/07-2020 Instacart Basket Analysis'

In [4]:
# Importing Data: customers
cust= pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [5]:
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
cust.shape

(206209, 10)

# 02. Wrangling

In [7]:
#rename surnam to last name
cust.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)

In [8]:
# rename STATE to state
cust.rename(columns = {'STATE' : 'State'}, inplace = True)

In [9]:
 cust.head()

,user_id,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [10]:
# rename n_dependants to dependants
cust.rename(columns = {'n_dependants' : 'dependants'}, inplace = True)

In [11]:
# check fam status: no name change necessary 
cust['fam_status'].value_counts(dropna = False)

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: fam_status, dtype: int64

In [12]:
cust.head()

,user_id,First Name,Last Name,Gender,State,Age,date_joined,dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [13]:
cust.describe()

,user_id,Age,dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


# 03. Data Quality and Consistency Checks

In [15]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  int64 
 1   First Name   194950 non-null  object
 2   Last Name    206209 non-null  object
 3   Gender       206209 non-null  object
 4   State        206209 non-null  object
 5   Age          206209 non-null  int64 
 6   date_joined  206209 non-null  object
 7   dependants   206209 non-null  int64 
 8   fam_status   206209 non-null  object
 9   income       206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


###### missing values in first name account for over 5% so it is best to just leave them as blank as we cannot use another method to fill in those names

In [19]:
# looking for mixed data types
for col in cust.columns.tolist():
  weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (cust[weird]) > 0:
    print (col)

First Name


##### First name has mixed data type due to missing values

In [20]:
# changing first name to string
cust['First Name'] = cust['First Name'].astype('str')

In [23]:
# finding duplicates
df_dups = cust[cust.duplicated()]

In [24]:
df_dups

,user_id,First Name,Last Name,Gender,State,Age,date_joined,dependants,fam_status,income


###### no dups found

# Merging data

In [25]:
# Importing Data: newest ords prods
opm = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_4.pkl'))

In [26]:
opm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                int64   
 2   order_number           int64   
 3   orders_day_of_week     int64   
 4   order_hour_of_day      int64   
 5   days_since_last_order  float64 
 6   product_id             int64   
 7   add_to_cart_order      int64   
 8   reordered              int64   
 9   product_name           object  
 10  aisle_id               int64   
 11  department_id          int64   
 12  prices                 float64 
 13  _merge                 category
 14  busiest_day            object  
 15  price_range_loc        object  
 16  busiest_days           object  
 17  busiest_period_of_day  object  
 18  max_order              int64   
 19  loyalty_flag           object  
 20  mean_price             float64 
 21  spending_flag          object

In [27]:
opm.shape

(32404859, 24)

In [28]:
cust.shape

(206209, 10)

In [32]:
cust['user_id'] = cust['user_id'].astype('str')

In [33]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  object
 1   First Name   206209 non-null  object
 2   Last Name    206209 non-null  object
 3   Gender       206209 non-null  object
 4   State        206209 non-null  object
 5   Age          206209 non-null  int64 
 6   date_joined  206209 non-null  object
 7   dependants   206209 non-null  int64 
 8   fam_status   206209 non-null  object
 9   income       206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


In [35]:
opm['user_id'] = opm['user_id'].astype('str')

In [36]:
opm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                object  
 2   order_number           int64   
 3   orders_day_of_week     int64   
 4   order_hour_of_day      int64   
 5   days_since_last_order  float64 
 6   product_id             int64   
 7   add_to_cart_order      int64   
 8   reordered              int64   
 9   product_name           object  
 10  aisle_id               int64   
 11  department_id          int64   
 12  prices                 float64 
 13  _merge                 category
 14  busiest_day            object  
 15  price_range_loc        object  
 16  busiest_days           object  
 17  busiest_period_of_day  object  
 18  max_order              int64   
 19  loyalty_flag           object  
 20  mean_price             float64 
 21  spending_flag          object

In [37]:
# merge opm and cust
cust_opm = opm.merge(cust, on = 'user_id')

In [38]:
#check 
cust_opm.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,First Name,Last Name,Gender,State,Age,date_joined,dependants,fam_status,income
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [39]:
cust_opm.shape

(32404859, 33)

In [41]:
cust_opm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 33 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                object  
 2   order_number           int64   
 3   orders_day_of_week     int64   
 4   order_hour_of_day      int64   
 5   days_since_last_order  float64 
 6   product_id             int64   
 7   add_to_cart_order      int64   
 8   reordered              int64   
 9   product_name           object  
 10  aisle_id               int64   
 11  department_id          int64   
 12  prices                 float64 
 13  _merge                 category
 14  busiest_day            object  
 15  price_range_loc        object  
 16  busiest_days           object  
 17  busiest_period_of_day  object  
 18  max_order              int64   
 19  loyalty_flag           object  
 20  mean_price             float64 
 21  spending_flag          object

# Export

In [42]:
#export merged df
cust_opm.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_cust_merge.pkl'))

In [43]:
#export cleaned customer df
cust.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customer.pkl'))